# Install the Custom Vision SDK and import the libraries

In [ ]:
#!pip install azure-cognitiveservices-vision-customvision
from azure.cognitiveservices.vision.customvision.training import CustomVisionTrainingClient
from azure.cognitiveservices.vision.customvision.training.models import ImageFileCreateEntry
import numpy as np

# Create Custom Vision Project

In [ ]:
# Replace with your key and endpoint
ENDPOINT = "https://<YourCustomVisionResouceName>.cognitiveservices.azure.com/"
training_key = "<YourKeyHere>"
trainer = CustomVisionTrainingClient(training_key, endpoint=ENDPOINT)

In [ ]:
# Create a new project
print ("Creating project...")
project = trainer.create_project("Bird Classification Test")
print("Project created!")

# Create tags list from file names

In [ ]:
#Create Tag List from folders in bird directory
import os
os.chdir('./bird_photos')
tags = [name for name in os.listdir('.') if os.path.isdir(name)]
print(tags)

# `createTag` function to create tag(s) in Azure

In [ ]:
def createTag(tag):
    result = trainer.create_tag(project.id, tag)
    print(f'{tag} create with id: {result}')
    return result.id


# `createImageList` function to create list of images to upload

In [ ]:
def createImageList(tag, tag_id):
    #set directory to current tag
    base_image_url = f"./{tag}/"
    photo_name_list = os.listdir(base_image_url)
    image_list = []
    for file_name in photo_name_list:
        with open(base_image_url+file_name, "rb") as image_contents:
            image_list.append(ImageFileCreateEntry(name=base_image_url+file_name, contents=image_contents.read(), tag_ids=[tag_id]))
    return image_list

# `uploadImageList` function to upload list of images

In [ ]:
def uploadImageList(image_list):  
    upload_result = trainer.create_images_from_files(project.id, images=image_list)
    if not upload_result.is_batch_successful:
        print("Image batch upload failed.")
        for image in upload_result.images:
            print("Image status: ", image.status)
        exit(-1)

# Loop thru tags and call each function to `createTag`, `createImageList`, then `uploadImageList` in batches of 25

In [ ]:
#get images names from directory
for tag in tags:
    tag_id = createTag(tag)
    print(f"tag creation done with tag id {tag_id}")
    image_list = createImageList(tag, tag_id)
    print("image_list created with length " + str(len(image_list)))
    
    #break list into lists of 25 and upload in batches
    for i in range(0, len(image_list), 25):
        batch = image_list[i:i + 25]
        print(f'Upload started for batch {i} total items {len(batch)} for tag {tag}...')
        uploadImageList(batch)
        print(f"Batch {i} Image upload completed. Total uploaded {len(batch)} for tag {tag}")

# Train Model

In [ ]:
import time

print ("Training...")
iteration = trainer.train_project(project.id)

while (iteration.status != "Completed"):
    iteration = trainer.get_iteration(project.id, iteration.id)
    print ("Training status: " + iteration.status)
    time.sleep(10)

# Publish the trained model to an endpoint

In [ ]:
# The iteration is now trained. Publish it to the project endpoint
#The prediction_resource_id can be found in the "<nameOfComputerVisionResource>-Prediction" resource in the portal
#Click on the resource then select the "properties" from the left nav. Copy the "Resource ID" and paste below.
prediction_resource_id = '/subscriptions/<subscription_id>/resourceGroups/CustomVision/providers/Microsoft.CognitiveServices/accounts/<nameOfComputerVisionResource>-Prediction'
trainer.publish_iteration(project.id, iteration.id, iteration.name, prediction_resource_id)
print ("Done!")

# Test the published endpoint

In [ ]:
from azure.cognitiveservices.vision.customvision.prediction import CustomVisionPredictionClient

# Now there is a trained endpoint that can be used to make a prediction
prediction_key = "<YourKeyHere>"
endpoint = "https://<UpdateWithYourEndpoint>-prediction.cognitiveservices.azure.com/"
base_image_url = "./"
# Example of default iteration name would be "Iteration1"
publish_iteration_name = "<YourIterationNameHere>"
# You can find the project id in the settings of the custom vision project in the portal.
project_id = "<YourProjectIdHere>"

predictor = CustomVisionPredictionClient(prediction_key, endpoint=endpoint)

with open(base_image_url + "bird_photos/American Crow/0ee2572a87514c3f97ff20b5d3eb9966.jpg", "rb") as image_contents:
    results = predictor.classify_image(
        project_id, publish_iteration_name, image_contents.read())

    # Display the results.
    for prediction in results.predictions:
        print("\t" + prediction.tag_name +
              ": {0:.2f}%".format(prediction.probability * 100))